# Eventdata und AMP-Berechnung für ROMY/RLAS

In [1]:
import os
import gc
import glob
import sys
import obspy as obs
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from functions.hilfsfunktionen import  transform_to_rtz
from obspy.geodetics import gps2dist_azimuth,locations2degrees
from pprint import pprint
from functions.add_distances_and_backazimuth import __add_distances_and_backazimuth
from functions.querrySeismoData import __querrySeismoData
from obspy import read
from scipy import fftpack
from andbro__fft import __fft
from obspy.taup import TauPyModel
from functions.get_octave_bands import __get_octave_bands

### Methods

In [2]:
def __makeplotspectraZsmooth(st, config,romy,freq_bands):
    st_in = st.select(component="Z")
    
    if len(st_in) == 0:
        print(" -> keine Z-Komponente gefunden.")
        return
    # Initialisierung
    tr1, tr2 = None, None

    max_amplitudes_spec1 = []
    max_amplitudes_spec2 = []
    result_dict = {
        "origin_time": str(origin_time),
        }   
    for tr in st_in:
        if romy == True:
            if tr.stats.station.upper() == "ROMY":
                tr1 = tr
                data1 = tr1.data
                spec1, ff1, ph1 = __fft(data1 ,tr.stats.delta, window=None, normalize=None)
                spec1_s = spec1#pd.Series(spec1).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
                for fmin, fmax in freq_bands:
                    mask = (ff1 >= fmin) & (ff1 <= fmax)
                    if np.any(mask):
                        perc95 = np.percentile(spec1_s[mask], 95)
                        max_amplitudes_spec1.append(perc95)
                    else:
                        max_amplitudes_spec1.append(np.nan)
                for i, center in enumerate(band_centers):
                     result_dict[f"amp1_{center:.4f}Hz"] = max_amplitudes_spec1[i]
    
        else:
            spec1,ff1,ph1 = None, None, None
        if tr.stats.station.upper() == "RLAS":
            tr2 = tr
            data2 = tr2.data
            spec2, ff2, ph2 = __fft(data2, tr.stats.delta, window=None, normalize=None)
            spec2_s = spec2#pd.Series(spec2).rolling(window=20, center=True).mean().fillna(method='bfill').fillna(method='ffill')
            max_amplitudes_spec2 = []
            for fmin, fmax in freq_bands:
                mask = (ff2 >= fmin) & (ff2 <= fmax)
                if np.any(mask):
                     perc95 = np.percentile(spec2_s[mask], 95)
                     max_amplitudes_spec2.append(perc95)
                else:
                     max_amplitudes_spec2.append(np.nan)
            for i, center in enumerate(band_centers):
                result_dict[f"amp2_{center:.4f}Hz"] = max_amplitudes_spec2[i]
    
    del st_in
    return result_dict

### Configurations

In [3]:
config = {}

# ROMY coordinates
config['ROMY_lon'] = 11.275501
config['ROMY_lat'] = 48.162941

# duration of event in seconds
config['duration'] = 7200

# frequency range for bandpass filter
config['fmin'] = 0.01 # in Hz   urspr. 0.01
config['fmax'] = 1 # in Hz    urspr. 0.1

# path for figures to store
config['outpath_figs'] = "C:/Bachelorarbeit/figures/acc/ROMYRLAS/"

# path for output data
config['outpath_data'] = "C:/Bachelorarbeit/data/waveformsROMYRLAS/"

# specify seed codes of stations that should be used for the analysis
config['seeds'] = ["BW.ROMY.10.BJZ", "BW.ROMY..BJU", "BW.ROMY..BJV", "BW.ROMY..BJW", # ringlaser ROMY
                   "BW.RLAS..BJZ", # ringlaser G
                  ]

# path to catalogs
config['path_to_catalog'] = "C:/Bachelorarbeit/data/catalogs/"

# catalog file
config['catalog'] = "ROMY_global_catalog_20190101_20250531.pkl"

## Load Events

In [4]:
events = pd.read_pickle(config['path_to_catalog']+config['catalog'])
events['origin'] = events.timestamp

In [5]:
# make sure only events with magnitude > 6 are considered
events = events[events.magnitude > 6]
print("Event number: ", events.shape[0])

Event number:  533


In [6]:
# avoid events that are too close to each other in time
events['elapsed_time'] = events.timestamp.diff()
events = events[events.elapsed_time > pd.Timedelta(minutes=60)]
print("Event number: ", events.shape[0])

Event number:  451


# RUN LOOP

In [7]:
errors = []
results_Z = []
i = 0
model = TauPyModel(model="prem")
fmin = config['fmin']
fmax = config['fmax']   
fraction_of_octave = 3  

f_lower, f_upper, f_center = __get_octave_bands(fmin, fmax, faction_of_octave=fraction_of_octave, plot=False)
freq_bands = [(round(fl, 5), round(fu, 5)) for fl, fu in zip(f_lower, f_upper)][2:] 
band_centers = [round(fc, 5) for fc in f_center][2:]
print(freq_bands)
duration = {
#   i: duration 
    13: 1400, 42: 1500, 219: 1300,372: 1500, 376: 1000, 377: 1500,387: 1500, 396: 1300,
    9: 3000, 23: 2000,31: 700,34: 1300,35: 850,38: 1300,40: 1400,55: 1700,70: 1200,
    74: 2100,93: 1000,98: 1000,110: 450, 148: 1400, 405: 2500,
    151: 1500, 173: 1000, 216: 1000,217: 1000, 220: 900, 221: 2500, 353: 1000, 356: 1500, 363: 1500}
start ={
    13: 1500,42: 1500,219:1700,372: 1500,376:1500, 377:1500, 387: 1550,396: 1700,397:1500,
    23: 2000,31: 1800,34: 1650,35: 1900,38: 2500,53: 2000,40: 2600,55: 1800,93: 1000,98: 2500,110: 200, 148: 1600, 
    151: 2500, 173: 1800,216: 1200, 217: 2500, 220: 2050,221: 1500,353: 1500,356: 1500, 363: 500,
    405: 1000}
    
good_events = {13,23,31,34,35,38,40,42,55,93,98,110,148,151,173,216,217,
                220,221,353,356,363,372,397,405,219,255,387,397,396,405}
good_events_RLAS = {56,70,71,78,80,88,92,93,100,109,152,164,174,181,183,212,243,244,246,247,390,409,417,429,433}
# loop over all events
for jj in range(events.shape[0]):
    if i not in good_events and i in good_events_RLAS:
        romy = False
    else:
        romy = True
    if not i in good_events and not i in good_events_RLAS:
        i+=1
        continue
    i+=1
    num = str(jj).rjust(3, "0")
    print(f"\n -> {num} {events.origin.iloc[jj]}")   
       
    try:
        event_name = str(events.origin.iloc[jj]).replace("-", "").replace(":", "").replace(" ", "_").split(".")[0]
    except:
        print(f" -> {num}: error for {events.origin.iloc[jj]}")
        
        continue

    # event metadata
    ev_lat = events.latitude.iloc[jj]
    ev_lon = events.longitude.iloc[jj]
    sta_lat = config['ROMY_lat']
    sta_lon = config['ROMY_lon']
    distance_m, az, backazimuth = gps2dist_azimuth(ev_lat, ev_lon, sta_lat, sta_lon)
    distance_km = distance_m / 1000
    ev_depth = events.depth.iloc[jj] / 1000
    origin_time = obs.UTCDateTime(events.origin.iloc[jj])
    distance_deg = locations2degrees(ev_lat, ev_lon, sta_lat, sta_lon)
    arrivals = model.get_travel_times(source_depth_in_km=ev_depth,
                                  distance_in_degree=distance_deg)
    
    if len(arrivals) == 0:
        print(f"Keine P-Ankunft für Event {jj} (Tiefe {ev_depth} km, Distanz {distance_deg}°)")
        starttime = origin_time
    else:
        p_arrival_time = arrivals[0].time 
        print(p_arrival_time)
        starttime = origin_time + p_arrival_time - 20

    try:
        config['tbeg'] = starttime + start.get(jj, 0)
        
        config['duration'] = duration.get(jj)
        
        if config['duration'] is None:
            if distance_km < 2000:
                config['duration'] = 1000
            elif 2000 < distance_km < 5000:
                config['duration'] = 1500
            elif 5000 < distance_km < 8000:
                config['duration'] = 2500
            elif 8000 < distance_km < 10000:
                config['duration'] = 3000
            elif 10000 < distance_km < 14000:
                config['duration'] = 4000
            elif 14000 < distance_km < 15000:
                config['duration'] = 4500
    
        config['tend'] = config['tbeg'] + config['duration']

    except Exception as e:
        config['tbeg'] = origin_time + start.get(jj, 0)
        config['tend'] = config['tbeg'] + 7200
        print(f"Fehler bei Index {jj}: {e}")

    st0 = obs.Stream()
    print(config['outpath_data']+f"{num}_{event_name}.mseed")
    print(os.path.isfile(config['outpath_data']+f"{num}_{event_name}.mseed"))
    if os.path.isfile(config['outpath_data']+f"{num}_{event_name}.mseed"):
        st0 = read(config['outpath_data']+f"{num}_{event_name}.mseed")
        data_vorhanden = True
    else:
        data_vorhanden = False
        print("keine Daten")
        continue
    # sort stream
    st0 = st0.sort()
    print(len(st0))
    st = st0.select(component="Z")
    if romy == True:
        if len(st) != 2:
            print("Keine 2 Z Komponenten")
            continue
    if romy == False:
        romy=False
    for tr in st0:
        if isinstance(tr.data, np.ma.MaskedArray):
            print(f" -> {tr.stats.channel} has masked data. Filled with zeros.")
            tr.data = tr.data.filled(fill_value=0)

    # preprocess
    print(" -> processing data stream ...")
    st1 = st0.copy()
    st1 = st1.detrend("linear")
    st1 = st1.taper(0.1)
    st1 = st1.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=4, zerophase=True)
    st1_rtz = st1
    # trim streams
    if config['tbeg'] > config['tend']:
        print(config['tbeg'], config['tend'])
        continue
    st1_rtz = st1_rtz.trim(config['tbeg'], config['tend'])
    st0 = st0.trim(config['tbeg'], config['tend'])
    for tr in st1_rtz:
        tr.differentiate()
    if not data_vorhanden:
        waveform_filename = f"{num}_{event_name}.mseed"
        if not os.path.isdir(config['outpath_data']):
            print("created: ", config['outpath_data'])
            os.makedirs(config['outpath_data'])

        try:
            st0.write(config['outpath_data'] + waveform_filename)
            print(f" -> stored at: {config['outpath_data'] + waveform_filename}")
        except Exception as e:
            print(f" -> error storing waveform: {e}")
            errors.append(f" -> error storing waveform: {e}")
            

    results_z = __makeplotspectraZsmooth(st1_rtz, config,romy,freq_bands)
    df = pd.DataFrame([results_z])
    z_result_path = config['outpath_figs'] + f"Z_results/{num}_{event_name}.csv"
    df.to_csv(z_result_path, index=False)
    del df, results_z, st0, st1, st1_rtz
    gc.collect()
# show errors
pprint(errors)


[(0.01413, 0.01778), (0.01778, 0.02239), (0.02239, 0.02818), (0.02818, 0.03548), (0.03548, 0.04467), (0.04467, 0.05623), (0.05623, 0.07079), (0.07079, 0.08913), (0.08913, 0.1122), (0.1122, 0.14125), (0.14125, 0.17783), (0.17783, 0.22387), (0.22387, 0.28184), (0.28184, 0.35481), (0.35481, 0.44668), (0.44668, 0.56234), (0.56234, 0.70795), (0.70795, 0.89125), (0.89125, 1.12202)]

 -> 013 2019-03-28 22:06:49.350000
715.3583652212435
C:/Bachelorarbeit/data/waveformsROMYRLAS/013_20190328_220649.mseed
True
5
 -> processing data stream ...

 -> 023 2019-05-14 12:58:26.610000
933.2369627434684
C:/Bachelorarbeit/data/waveformsROMYRLAS/023_20190514_125826.mseed
True
5
 -> processing data stream ...

 -> 031 2019-06-18 13:22:21.040000
736.306070473979
C:/Bachelorarbeit/data/waveformsROMYRLAS/031_20190618_132221.mseed
True
5
 -> processing data stream ...

 -> 034 2019-06-25 09:05:41.840000
688.7247178046679
C:/Bachelorarbeit/data/waveformsROMYRLAS/034_20190625_090541.mseed
True
5
 -> processing da

In [8]:
# Z-Komponente
z_files = glob.glob(config['outpath_figs'] + "Z_results/*.csv")
df_all_Z = pd.concat([pd.read_csv(f) for f in z_files], ignore_index=True)
df_all_Z.to_csv(config['outpath_figs'] + "amp_ergebnisse_Z.csv", index=False)